In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
print(train_df)

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df.info()

In [ ]:
train_df['date'] = pd.to_datetime(train_df['date'], format="%d.%m.%Y")
print(train_df)

In [ ]:
train_df['date'].max()

In [ ]:
train_df['date'].min()

In [ ]:
train_df['day'] = train_df['date'].dt.day
train_df['month'] = train_df['date'].dt.month
train_df['year'] = train_df['date'].dt.year
train_df['month_year'] = pd.to_datetime(train_df['date']).dt.to_period('M')
print(train_df.sort_values(by = ['date']))

In [ ]:
train_df[train_df["item_cnt_day"] <= 0]


In [ ]:
train_df[train_df["item_cnt_day"] > 2000]

In [ ]:
train_df["item_cnt_day"].max()

In [ ]:
train_df["item_cnt_day"].min()

In [ ]:
train_df[train_df["item_price"] <= 0]


In [ ]:
train_df[train_df["item_price"] > 100000]

In [ ]:
train_df["item_price"].max()

In [ ]:
train_df["item_price"].min()

In [ ]:
train_df = train_df[(train_df["item_price"] > 0) & (train_df["item_price"] < 2000)]

In [ ]:
train_df = train_df[(train_df["item_cnt_day"] > 0) & (train_df["item_cnt_day"] < 100000)]

In [ ]:
print(train_df)

In [ ]:
test_df = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
print(test_df)

In [ ]:
test_df['date_block_num'] = 34
test_data =test_df.drop(['ID'], axis=1)
print(test_data)

In [ ]:
set(test_data['shop_id'].unique()).difference(set(train_df['shop_id'].unique()))


In [ ]:
train_df['shop_id'].unique()

In [ ]:
shops_data = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
shops_data = shops_data.sort_values(by=['shop_name'])

print(shops_data)

In [ ]:
shops_data[['city','shop_name']] = shops_data['shop_name'].str.split(' ', 1, expand=True, )

print(shops_data)

In [ ]:
items_data = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
print(items_data)

In [ ]:
items_data.item_category_id.value_counts() 

In [ ]:
All_df = train_df.merge(shops_data, left_on='shop_id', right_on='shop_id')
All_df = All_df.merge(items_data, left_on='item_id', right_on='item_id')
print(All_df)

In [ ]:
item_cnt_shop_id = All_df.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_day': 'sum','item_price': np.mean}).reset_index()
print(item_cnt_shop_id)

In [ ]:
item_cnt_city = All_df.groupby(['date_block_num', 'city']).agg({'item_cnt_day': 'sum','item_price': np.mean}).reset_index()
print(item_cnt_city)

In [ ]:
item_cnt_item = All_df.groupby(['date_block_num', 'item_id']).agg({'item_cnt_day': 'sum','item_price': np.mean}).reset_index()
print(item_cnt_item)

In [ ]:
item_cnt_item_cat = All_df.groupby(['date_block_num', 'shop_id', 'item_category_id']).agg({'item_cnt_day': np.mean,'item_price': np.mean}).reset_index()
print(item_cnt_item_cat)

In [ ]:
sales_monthly = All_df.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': 'sum','item_price': np.mean}).reset_index()
print(sales_monthly)

In [ ]:
sales_monthly_df = sales_monthly[['shop_id', 'item_id', 'date_block_num', 'item_cnt_day']]
print(sales_monthly_df)

In [ ]:
test_df1 = test_data.merge(sales_monthly, on=['shop_id', 'item_id'], how='left', indicator=True)
print(test_df1)

In [ ]:
df_test_only = test_df1[test_df1['_merge']== 'left_only'] 
df_test_only = df_test_only.drop(['date_block_num_y',  'item_cnt_day', 'item_price', '_merge'], axis = 1)
print(df_test_only)

In [ ]:
df_test_only = df_test_only.merge(items_data, left_on='item_id', right_on='item_id')
df_test_only = df_test_only.drop(['item_name'], axis=1)
print(df_test_only)

In [ ]:
test_merged_df = df_test_only.merge(item_cnt_item_cat,  left_on=['shop_id', 'item_category_id'], right_on=['shop_id', 'item_category_id'])

test_merged_df = test_merged_df.drop(['date_block_num_x','item_category_id'], axis =1)
test_merged_df = test_merged_df.drop(['item_price'], axis =1)
print(test_merged_df)

In [ ]:

df_total = pd.concat([sales_monthly_df, test_merged_df], ignore_index=True)


print(df_total)

In [ ]:
df_total_34 = pd.concat([df_total, test_data], ignore_index=True)

df_total_34 = df_total_34.fillna(0)

print(df_total_34)

In [ ]:

tmp2 = df_total_34.copy()
for i in range(1,11):
    tmp2[str(i)+'_month_Ago_'] = tmp2.groupby(['shop_id', 'item_id'])['item_cnt_day'].shift(i) 
    
 
            

tmp2 = tmp2.dropna().reset_index(drop=True)

                       
print(tmp2)    
       

In [ ]:
tmp2[tmp2['date_block_num']== 34 ]

In [ ]:
for i in range(1, 35):

            train = tmp2[tmp2['date_block_num'] < i]
            val   = tmp2[tmp2['date_block_num'] == i]

            X_train, X_test = train.drop(['item_cnt_day','shop_id', 'item_id'], axis=1), val.drop(['item_cnt_day', 'shop_id', 'item_id'], axis=1)
            y_train, y_test = train['item_cnt_day'].values, val['item_cnt_day'].values

            
            
            #ield X_train, X_test, y_train, y_test

    
print(X_train)
print(y_train)

print(X_test)
print(y_test)        

In [ ]:
from numpy import absolute

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
# load the dataset

model3 = XGBRegressor()
# define model evaluation method
#cv = RepeatedKFold(n_splits=3, n_repeats=3, random_state=1)
# evaluate model
#scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force scores to be positive
#scores = absolute(scores)
#print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

In [ ]:
model3.fit(X_train, y_train)

In [ ]:
yhat3 = model3.predict(X_test)
print(yhat3)

In [ ]:
from math import sqrt
from sklearn.metrics import mean_squared_error
print('Root mean squared error(RMSE): %.2f'% sqrt(mean_squared_error(y_test,yhat3)))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

model4 = RandomForestRegressor(n_estimators=50, max_depth=2, random_state=0)
model4.fit(X_train, y_train)




In [ ]:
yhat4 = model4.predict(X_test)
print(yhat4)

In [ ]:
from math import sqrt
print('Root mean squared error(RMSE): %.2f'% sqrt(mean_squared_error(y_test,yhat4)))

In [ ]:
subm_df = tmp2[tmp2['date_block_num']== 34 ]
print(subm_df)

In [ ]:
subm_df['Prediction'] = yhat3
subm_df = subm_df[['shop_id', 'item_id', 'Prediction']]
print(subm_df)

In [ ]:
submision = test_data.merge(subm_df, on=['shop_id', 'item_id'], how='outer', indicator=True)
submision =submision.drop_duplicates(subset=['shop_id', 'item_id'], keep="first").replace(np.nan, yhat3.mean())
print(submision)

In [ ]:
df = pd.DataFrame(submision['Prediction']).reset_index()
df = df.rename(columns={"index": "ID", 'Prediction': "item_cnt_month"})
print(df)

In [ ]:
print(test_df['ID'])

In [ ]:
df.to_csv('prediction_file.csv', index=False)

###### 